In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
url = "https://raw.githubusercontent.com/martin2urban/pythonProject1/master/phiSlim30cols.csv"
columns= ['PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID']
my_types = {
    "PhiAcc": str,
#   "ProtId": str,
#   "PathSpecies": str,
#  "PathSpeciesTaxID": int,

}
phi2 = pd.read_csv(
    "phiSlim30cols.csv",
    usecols=columns,
#    dtype=my_types,
    header=0,
)
print(phi2.columns)
phi2.describe(include='all')

Index(['PhiAcc', 'ProtId', 'PathSpeciesTaxID', 'PathSpecies'], dtype='object')


,PhiAcc,ProtId,PathSpeciesTaxID,PathSpecies
count,20951,20951,20951,20951
unique,12144,9378,286,286
top,PHI:8380,no data found,5518,Fusarium graminearum
freq,69,2426,1800,1800


In [7]:
#How do I convert the columns 'PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID' to strings?
#
columns = ['PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID']
non_string_values_per_column = {}
for column in phi2.columns:
    non_string_values = set(phi2[column].astype(str).str.findall(r'[^a-zA-Z\s\d:]').sum())
    non_string_values_per_column[column] = non_string_values

for column, non_string_values in non_string_values_per_column.items():
    print(f"Non-string values in column '{column}':")
    for value in non_string_values:
        print(value)
    print()



Non-string values in column 'PhiAcc':
_

Non-string values in column 'ProtId':
-
.

Non-string values in column 'PathSpeciesTaxID':

Non-string values in column 'PathSpecies':
-
.
'



In [8]:
url = "https://raw.githubusercontent.com/martin2urban/pythonProject1/master/phiSlim30cols.csv"
columns= ['PhiAcc', 'ProtId', 'PathSpecies', 'PathSpeciesTaxID']
my_types = {
    "PhiAcc": str,
#   "ProtId": str,
#   "PathSpecies": str,
#  "PathSpeciesTaxID": int,

}
phi2 = pd.read_csv(
    "phiSlim30cols.csv",
    usecols=columns,
#    dtype=my_types,
    header=0,
)

In [9]:
phiSlim = pd.read_csv('phiSlim30cols.csv', header=0)
phiSlim.describe()

,PhiAcc,ProtDb,ProtId,GeneDb,LocusId,SeqStrain,GenName,MulMut,PathSpeciesTaxID,PathSpecies,...,Function,GO,Phenotype,Vegetative_spores,Sexual_spores,In_vitro_growth,Spore_germination,Literature_Id,Year_published,Chemical.group
count,20951,18525,20951,18892,19321,18036,20951,5176,20951,20951,...,2324,2311,615,1902,8476,1930,19523,20030,20951,1
unique,12144,3,9378,19,10214,1603,10547,3334,286,286,...,137,3,7,24,65,23,4,2885,159,1
top,PHI:8380,Uniprot,no data found,Genbank,no data found,PH-1,ToxB (AAO73337),PHI:8380,5518,Fusarium graminearum,...,no data found,no data found,no data found,wild type,reduced,wild type,no,ee2443@snu.ac.kr,42278,Anti infective-comments
freq,69,18339,2426,15696,272,1514,69,69,1800,1800,...,1508,1495,281,922,2639,750,19304,709,594,1


In [10]:
# tranform all string data in phiSlim to lowercase
phiSlim = phiSlim.applymap(lambda x: x.lower() if isinstance(x, str) else x)
phiSlim


,PhiAcc,ProtDb,ProtId,GeneDb,LocusId,SeqStrain,GenName,MulMut,PathSpeciesTaxID,PathSpecies,...,Function,GO,Phenotype,Vegetative_spores,Sexual_spores,In_vitro_growth,Spore_germination,Literature_Id,Year_published,Chemical.group
0,phi_molconn_id,protein id source,protein id,gene id source,gene id,sequence strain,gene,multiple mutation,pathogen id,pathogen species,...,go annotation,database,mating defect,sexual spores,invitro growth,spore germination,essential gene,author email,curation date,anti infective-comments
1,phi:3,uniprot,p26215,embl,aaa79885,sb111,pgn1,NaN,5017,bipolaris zeicola,...,go:0004650,go,NaN,NaN,NaN,NaN,NaN,NaN,38476,NaN
2,phi:7,uniprot,p22287,embl,caa42824,race 5,avr9,NaN,5499,fulvia fulva,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38476,NaN
3,phi:12,uniprot,q01886,embl,aaa33023,sb111,hts1,NaN,5017,bipolaris zeicola,...,"go:0009405, imp",go,NaN,NaN,NaN,NaN,NaN,NaN,38476,NaN
4,phi:14,uniprot,p0c017,embl,aab09711,clinical isolate,ade2,NaN,5207,cryptococcus neoformans,...,"go:0009405, imp",go,NaN,NaN,NaN,NaN,NaN,NaN,38476,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20946,phi:123548,uniprot,g4msa2,genbank,eha58360,70-15,moade8 (mgg_13813),NaN,318829,magnaporthe oryzae,...,NaN,NaN,NaN,NaN,defective,NaN,no,xhliu@zju.edu.cn,22-oct,NaN
20947,phi:123548,uniprot,g4msa2,genbank,eha58360,70-15,moade8 (mgg_13813),NaN,318829,magnaporthe oryzae,...,NaN,NaN,NaN,NaN,defective,NaN,no,xhliu@zju.edu.cn,22-oct,NaN
20948,phi:123549,uniprot,g4mu94,genbank,eha53975,70-15,"moade5,7 (mgg_11343)",NaN,318829,magnaporthe oryzae,...,NaN,NaN,NaN,NaN,NaN,NaN,no,xhliu@zju.edu.cn,22-oct,NaN
20949,phi:123550,uniprot,g4nb68,genbank,eha48830,70-15,moade6 (mgg_11343),NaN,318829,magnaporthe oryzae,...,NaN,NaN,NaN,NaN,NaN,NaN,no,xhliu@zju.edu.cn,22-oct,NaN


In [39]:
# make stats-counts report
# tabulate tally: specis x genes x interactions
# Group by 'PathSpecies' and calculate distinct 'PhiAcc' count and count of rows
data = phiSlim.groupby('PathSpecies').agg({'PhiAcc': 'nunique', 'PathSpecies': 'size'}).rename(columns={'PhiAcc': 'genes', 'PathSpecies': 'PHI.base.accession'}).reset_index()

# Sort the DataFrame by 'PHI.base.accession' in descending order
data = data.sort_values('PHI.base.accession', ascending=False)

# Rename 'PHI.base.accession' column to 'interactions'
data = data.rename(columns={'PHI.base.accession': 'interactions'})

# Write the DataFrame to a CSV file
data.to_csv(pd.Timestamp.today().strftime('%Y-%m-%d') +"phiSlim_speciesBYpathogen_geneBYinteraction_tally.csv", index=False, na_rep="")


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, 111 to 153
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PathSpecies   286 non-null    object
 1   genes         286 non-null    int64 
 2   interactions  286 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 8.9+ KB


In [19]:
data2


,0,1,2,3
111,fusarium,graminearum,None,None
146,magnaporthe,oryzae,None,None
224,salmonella,enterica,None,None
217,ralstonia,solanacearum,None,None
48,candida,albicans,None,None
...,...,...,...,...
184,penicillium,italicum,None,None
175,pathogen,species,None,None
156,muricauda,olearia,None,None
144,lonsdalea,quercina,None,None


In [40]:
# tabulate tally: genus x genes x interactions
# Split data "species name
data2 = data['PathSpecies'].str.split(' ', expand=True).rename(columns={0: 'genus', 1: 'species'})
data2 = data2.iloc[:, [0, 1]]
# Group by genus and count distinct species
data3 = data2.groupby('genus').agg(Species=('species', 'nunique')).reset_index()
# Sort by descending species count
data3 = data3.sort_values('Species', ascending=False)
# Write to CSV
data3.to_csv(pd.Timestamp.today().strftime('%Y-%m-%d') +'_phiSlim_genus_no_Species.csv', index=False)

In [35]:
data3


,genus,Species
55,fusarium,12
131,streptococcus,10
34,colletotrichum,10
109,pseudomonas,9
145,xanthomonas,9
...,...,...
59,grosmannia,1
57,glaesserella,1
56,gaeumannomyces,1
54,fulvia,1


In [54]:
columns = ['PhiAcc', 'PathSpecies', 'ProtId', 'HostSpeciesTaxId', 'Literature_Id', 'DiseaseName']
aggregations = {'PhiAcc': 'count', 'PathSpecies': 'nunique', 'ProtId': 'nunique', 'HostSpeciesTaxId': 'nunique', 'Literature_Id': 'nunique', 'DiseaseName': 'nunique'}

global_data = phiSlim[columns].agg(aggregations).reset_index()



In [56]:
global_data

,index,0
0,PhiAcc,20951
1,PathSpecies,286
2,ProtId,9378
3,HostSpeciesTaxId,237
4,Literature_Id,2833
5,DiseaseName,541
